In [ ]:
# header / imports
import sc3nb as scn
import time

In [ ]:
sc = scn.startup()

In [ ]:
# example objects which are used later
buffer = sc.Buffer().load_file("./my_recording.wav")
pbExample = sc.SynthDef(name="pbexample", definition="""{ |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
                    Out.ar(out,
                        PlayBuf.ar(2, bufnum, rate*BufRateScale.kr(bufnum), loop: 0, 
                                   doneAction: Done.freeSelf)!2
                )
            }""").create()

# Synth Definition
* SynthDef is a python class in the sc3nb package to interface with synthdefs on the SuperCollider3 server.
* A synthdef can be created with sc3nb.SynthDef(sc)
* The constructor is also invoked by the method sc3nb.SC.SynthDef(), a method of a booted SC instance, i.e. 
        synthDef = sc.SynthDef()
  to allocate a synthdef on the sound server booted from the SC instance.
* On synthdef methods, the synthdef instance is returned so that subsequent synthdef methods can be directly invoked
* This notebook demonstrates with some examples how to work with sc3nb.Buffer

## Create a simple SynthDef instance
Create a new synthdef instance in python. You can create it later on the sc server

In [ ]:
simpleSynthDef = sc.SynthDef(name="synthDef", definition="""{ |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
                    Out.ar(out,
                        PlayBuf.ar(2, bufnum, rate*BufRateScale.kr(bufnum), loop: ^loop, 
                                   doneAction: Done.freeSelf)!2
                )
            }""")
simpleSynthDef

## Create a SynthDef with flexible content
In some cases you want to have multiple synthdefs with only a small difference in the synthdef. For this case you can create a general synthdef instance with placeholders. Placerholders are written in the mustache syntax.

        {{placeholder}}
You can replace the placeholder in further steps with the real content and create in a easy way multiple synthdefs

In [ ]:
complexSynthDef = sc.SynthDef(name="synthDef", definition="""{ |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
                    Out.ar(out,
                        PlayBuf.ar({{channel}}, bufnum, rate*BufRateScale.kr(bufnum), loop: ^loop, 
                                   doneAction: Done.freeSelf)!2
                )
            }""")
complexSynthDef

### Modify flexible content

In [ ]:
flexibleSynthDef = sc.SynthDef(name="flexibleSynth", definition="""Hello {{world}}""")
flexibleSynthDef.set_context("world", "mars")

You can also do it n-times in a nested way:

In [ ]:
flexibleSynthDef = sc.SynthDef(name="flexibleSynth", definition="""Hello {{content}}""")
flexibleSynthDef.set_context("content", "mars {{content}}")
flexibleSynthDef.set_context("content", "and jupiter")

### Modify multiple flexible contents at once

In [ ]:
multiflexSynthDef = sc.SynthDef(name="multiflexibleSynth", definition="""Hello {{var1}} and  {{var2}}""")
multiflexSynthDef.set_contexts({"var1" : "ladies", "var2": "gentlemen"})

### Remove unused placeholders

In [ ]:
placeholderSynthDef = sc.SynthDef(name="placeholderSynth", definition="""Hello world! {{content}}""")
placeholderSynthDef.unset_remaining()

The definition is now complete -> lets create it in SC:
You can also add pyvars here if you want to modify the rendered query at the end.

## Reset a modified synth def to default value

In [ ]:
resetSynthDef = sc.SynthDef(name="resetSynth", definition="""Hello {{var1}}""")
resetSynthDef.set_context("var1", "world")

In [ ]:
resetSynthDef.reset()

## Create a synthdef in sc

In [ ]:
complexSynthDef = sc.SynthDef(name="synthDef", definition="""{ |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
                    Out.ar(out,
                        PlayBuf.ar({{channel}}, bufnum, rate*BufRateScale.kr(bufnum), loop: 0, 
                                   doneAction: Done.freeSelf)!2
                )
            }""")
complexSynthDef.set_context("channel", 2)

In [ ]:
complexSynthDef.create()

The result of create() will be the name of the created synthdef. The synthdef instance will not reset after a create:

In [ ]:
complexSynthDef

### Create a synthdef in sc and reset it afterwards

In [ ]:
complexSynthDef = sc.SynthDef(name="synthDef", definition="""{ |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
                    Out.ar(out,
                        PlayBuf.ar({{channel}}, bufnum, rate*BufRateScale.kr(bufnum), loop: 0, 
                                   doneAction: Done.freeSelf)!2
                )
            }""")
complexSynthDef.set_context("channel", 2)

In [ ]:
complexSynthDef.create_and_reset()

SynthDef will be resetted after the creation:

In [ ]:
complexSynthDef

### Create a synthdef in sc with PyVars

In [ ]:
complexSynthDef = sc.SynthDef(name="synthDef", definition="""{ |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
                    Out.ar(out,
                        PlayBuf.ar({{channel}}, bufnum, rate*BufRateScale.kr(bufnum), loop: ^loop, 
                                   doneAction: Done.freeSelf)!2
                )
            }""")
complexSynthDef.set_context("channel", 2)

In [ ]:
complexSynthDef.create(pyvars={"loop": 1})

## Delete a synth def in sc

In [ ]:
synthDef = sc.SynthDef(name="synthDef", definition="""{ |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
                    Out.ar(out,
                        PlayBuf.ar(2, bufnum, rate*BufRateScale.kr(bufnum), loop: 0, 
                                   doneAction: Done.freeSelf)!2
                )
            }""")
synthDefName = synthDef.create()

In [ ]:
synthDef.free(synthDefName)

## Tip: Create multiple synthdefs
In some cases you want to create many SynthDefs with only a small change. You can use the SynthDefs object multiple time to do this. Here we want to create playbuf synthdefs for 1 to 10 channels: 

In [ ]:
pbSynths = sc.SynthDef(name="pb", definition="""{ |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
                    Out.ar(out,
                        PlayBuf.ar({{channel}}, bufnum, rate*BufRateScale.kr(bufnum), loop: 0, 
                                   doneAction: Done.freeSelf)!2
                )
            }""")

In [ ]:
synthPlaybufs = {}
for channel in [1,2,3,4,5,6,7,8,9,10]:
    synthPlaybufs[channel] = pbSynths.set_context("channel", channel).create_and_reset()
synthPlaybufs



# Synth

## Create a new synth:

In [ ]:
synth = sc.Synth(name=pbExample, args={"bufnum": buffer.bufnum})

## Restart a synth

In [ ]:
synth = sc.Synth(name=pbExample, args={"bufnum": buffer.bufnum})
time.sleep(5)
synth.restart()

## Pause a synth

In [ ]:
synth = sc.Synth(name=pbExample, args={"bufnum": buffer.bufnum})
time.sleep(0.2)
synth.pause()

## Run a paused synth

In [ ]:
synth = sc.Synth(name=pbExample, args={"bufnum": buffer.bufnum})
time.sleep(0.5)
synth.pause()
time.sleep(1)
synth.run()

## Set a control to a synth

In [ ]:
synth = sc.Synth(name=pbExample, args={"bufnum": buffer.bufnum})
synth.set("loop", 1)

## Destroy a running synth

In [ ]:
synth = sc.Synth(name=pbExample, args={"bufnum": buffer.bufnum})
time.sleep(0.5)
synth.free()